In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import numpy as np
import json
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:


import torchvision.transforms as transforms

class DenoisingDataset(Dataset):
    def __init__(self, noisy_dir, clean_dir, transform=None  , json_path : str = None):
        self.noisy_dir = noisy_dir
        self.clean_dir = clean_dir
        self.transform = transform
        with open(file = json_path , mode = "r") as fp:
            self.image_files = json.load(fp = fp)
        self.resize_transform = transforms.Resize((256, 256))


    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # some minor tweaks
        noisy_image_path = os.path.join(self.noisy_dir, self.image_files[idx]["path_gt"])
        clean_image_path = os.path.join(self.clean_dir, self.image_files[idx]["path_lq"].replace("X4/" , ""))

        noisy_image = Image.open(noisy_image_path).convert("L")
        clean_image = Image.open(clean_image_path).convert("L")

        noisy_image = noisy_image.convert("RGB")
        clean_image = clean_image.convert("RGB")

        noisy_image = self.resize_transform(noisy_image)
        clean_image = self.resize_transform(clean_image)

        if self.transform:
            noisy_image = self.transform(noisy_image)
            clean_image = self.transform(clean_image)
        return noisy_image, clean_image


In [3]:
from torch.utils.data import random_split
batch_size = 16
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize between [-1,1]
])

full_dataset = DenoisingDataset(
        noisy_dir="/kaggle/input/image-dataset-collection/HR",
        clean_dir="/kaggle/input/image-dataset-collection/train",
        transform=transform,
        json_path = "/kaggle/input/image-mapping/train_X4.json"
    )
    
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
    
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
train_loader_x = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader_x = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(len(train_loader_x))
print(len(val_loader_x))


4250
1063


In [4]:
from torch.utils.data import Subset
import numpy as np


def get_subset_loader(dataset, percentage=0.1, batch_size=16, shuffle=True, num_workers=4):
    dataset_size = len(dataset)
    subset_size = int(dataset_size * percentage)
    indices = np.random.choice(dataset_size, subset_size, replace=False)  # Randomly sample 10% of indices
    subset = Subset(dataset, indices)
    return torch.utils.data.DataLoader(subset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)  , subset

# Assuming train_loader and val_loader have dataset attributes
train_subset_loader , train_subset = get_subset_loader(train_loader_x.dataset, percentage=0.8, batch_size=16)
val_subset_loader , val_subset = get_subset_loader(val_loader_x.dataset, percentage=0.05, batch_size=16)

print(len(train_subset_loader))
print(len(val_subset_loader))
print(len(train_subset))
print(len(val_subset))

3400
54
54393
849


In [10]:


import os
import time
import torch
from torch import nn
import torch.utils.data as td
from abc import ABC, abstractmethod


class NeuralNetwork(nn.Module, ABC):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

    @property
    def device(self):
        return next(self.parameters()).device

    def named_parameters(self, recurse=True):
        nps = nn.Module.named_parameters(self)
        for name, param in nps:
            if not param.requires_grad:
                continue
            yield name, param

    @abstractmethod
    def forward(self, x):
        pass

    @abstractmethod
    def criterion(self, y, d):
        pass


class StatsManager:
    def __init__(self):
        self.init()

    def __repr__(self):
        return self.__class__.__name__

    def init(self):
        self.running_loss = 0
        self.number_update = 0

    def accumulate(self, loss, x=None, y=None, d=None):
        self.running_loss += loss
        self.number_update += 1

    def summarize(self):
        return self.running_loss / self.number_update


class Experiment:
    def __init__(self, net, train_set, val_set, optimizer, stats_manager,
                 output_dir=None, batch_size=16, perform_validation_during_training=False):
        train_loader = td.DataLoader(train_set, batch_size=batch_size, shuffle=True,
                                     drop_last=True, pin_memory=True)
        val_loader = td.DataLoader(val_set, batch_size=batch_size, shuffle=False,
                                   drop_last=True, pin_memory=True)
        history = []
        if output_dir is None:
            output_dir = 'experiment_{}'.format(time.time())
        os.makedirs(output_dir, exist_ok=True)
        checkpoint_path = os.path.join(output_dir, "checkpoint.pth.tar")
        config_path = os.path.join(output_dir, "config.txt")
        locs = {k: v for k, v in locals().items() if k != 'self'}
        self.__dict__.update(locs)
        if os.path.isfile(config_path):
            with open(config_path, 'r') as f:
                if f.read().strip() != repr(self):
                    raise ValueError("Cannot create this experiment: checkpoint conflict.")
            self.load()
        else:
            self.save()

    @property
    def epoch(self):
        return len(self.history)

    def setting(self):
        return {'Net': self.net,
                'TrainSet': self.train_set,
                'ValSet': self.val_set,
                'Optimizer': self.optimizer,
                'StatsManager': self.stats_manager,
                'BatchSize': self.batch_size,
                'PerformValidationDuringTraining': self.perform_validation_during_training}

    def __repr__(self):
        return '\n'.join('{}({})'.format(k, v) for k, v in self.setting().items())

    def state_dict(self):
        return {'Net': self.net.state_dict(),
                'Optimizer': self.optimizer.state_dict(),
                'History': self.history}

    def load_state_dict(self, checkpoint):
        self.net.load_state_dict(checkpoint['Net'])
        self.optimizer.load_state_dict(checkpoint['Optimizer'])
        self.history = checkpoint['History']
        for state in self.optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor):
                    state[k] = v.to(self.net.device)

    def save(self):
        torch.save(self.state_dict(), self.checkpoint_path)
        with open(self.config_path, 'w') as f:
            print(self, file=f)

    def load(self):
        checkpoint = torch.load(self.checkpoint_path, map_location=self.net.device)
        self.load_state_dict(checkpoint)
        del checkpoint

    def run(self, num_epochs, plot=None):
        self.net.train()
        self.stats_manager.init()
        start_epoch = self.epoch
        print("Start/Continue training from epoch {}".format(start_epoch))
        if plot:
            plot(self)
        for epoch in range(start_epoch, num_epochs):
            s = time.time()
            self.stats_manager.init()
            print(f"Length of train loader is {len(self.train_loader)}")
            for x, d in self.train_loader:
                x, d = x.to(self.net.device), d.to(self.net.device)
                self.optimizer.zero_grad()
                y = self.net.forward(x)
                loss = self.net.criterion(y, d)
                loss.backward()
                self.optimizer.step()
                # print(f"loss is {loss}")
                with torch.no_grad():
                    self.stats_manager.accumulate(loss.item(), x, y, d)
            if not self.perform_validation_during_training:
                self.history.append(self.stats_manager.summarize())
                print("Epoch {} | Time: {:.2f}s | Training Loss: {:.6f}".format(
                    epoch, time.time() - s, self.history[-1]))

            else:
                self.history.append((self.stats_manager.summarize(), self.evaluate()))
                # print("Epoch {} | Time: {:.2f}s | Training Loss: {:.6f} | Evaluation Loss: {:.6f}".format(
                #     self.epoch, time.time() - s, self.history[-1][0], self.history[-1][1]))

                print("Epoch {} | Time: {:.2f}s | Training Loss: {:.6f} | Evaluation Loss: {:.6f}".format(
                        epoch, time.time() - s, self.history[-1][0]['loss'], self.history[-1][1]['loss']))
            self.save()
            if plot:
                plot(self)
        print("Finish training for {} epochs".format(num_epochs))

    def evaluate(self):
        self.stats_manager.init()
        self.net.eval()
        with torch.no_grad():
            for x, d in self.val_loader:
                x, d = x.to(self.net.device), d.to(self.net.device)
                y = self.net.forward(x)
                loss = self.net.criterion(y, d)
                self.stats_manager.accumulate(loss.item(), x, y, d)
        self.net.train()
        return self.stats_manager.summarize()


In [18]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn


def imshow(image, ax=plt):
    image = image.to('cpu').numpy()
    image = np.moveaxis(image, [0, 1, 2], [2, 0, 1])
    image = (image + 1) / 2
    image[image < 0] = 0
    image[image > 1] = 1
    h = ax.imshow(image)
    ax.axis('off')
    return h


def plot(exp, fig, axes, noisy, visu_rate=2):
    if exp.epoch % visu_rate != 0:
        return
    with torch.no_grad():
        denoised = exp.net(noisy[None].to(exp.net.device))[0]
    axes[0][0].clear()
    axes[0][1].clear()
    axes[1][0].clear()
    axes[1][1].clear()
    imshow(noisy, ax=axes[0][0])
    axes[0][0].set_title('Noisy image')

    imshow(denoised, ax=axes[0][1])
    axes[0][1].set_title('Denoised image')

    axes[1][0].plot([exp.history[k][0]['loss']
                     for k in range(exp.epoch)], label='training loss')
    axes[1][0].set_ylabel('Loss')
    axes[1][0].set_xlabel('Epoch')
    axes[1][0].legend()

    axes[1][1].plot([exp.history[k][0]['PSNR']
                     for k in range(exp.epoch)], label='training psnr')
    axes[1][1].set_ylabel('PSNR')
    axes[1][1].set_xlabel('Epoch')
    axes[1][1].legend()

    plt.tight_layout()
    fig.canvas.draw()


class NNRegressor(NeuralNetwork):

    def __init__(self):
        super(NNRegressor, self).__init__()
        self.mse = nn.MSELoss()

    def criterion(self, y, d):
        return self.mse(y, d)


class DenoisingStatsManager(StatsManager):

    def __init__(self):
        super(DenoisingStatsManager, self).__init__()

    def init(self):
        super(DenoisingStatsManager, self).init()
        self.running_psnr = 0

    def accumulate(self, loss, x, y, d):
        super(DenoisingStatsManager, self).accumulate(loss, x, y, d)
        n = x.shape[0] * x.shape[1] * x.shape[2] * x.shape[3]
        self.running_psnr += 10*torch.log10(4*n/(torch.norm(y-d)**2))

    def summarize(self):
        loss = super(DenoisingStatsManager, self).summarize()
        psnr = self.running_psnr / self.number_update
        print(f"running psrn is {psnr}")
        return {'loss': loss, 'PSNR': psnr.cpu()}

In [22]:
import os
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import torch.utils.data as td
import torchvision as tv
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt


class DnCNN(NNRegressor):

    def __init__(self, D, C=64):
        super(DnCNN, self).__init__()
        self.D = D

        # convolution layers
        self.conv = nn.ModuleList()
        self.conv.append(nn.Conv2d(3, C, 3, padding=1))
        self.conv.extend([nn.Conv2d(C, C, 3, padding=1) for _ in range(D)])
        self.conv.append(nn.Conv2d(C, 3, 3, padding=1))
        # apply He's initialization
        for i in range(len(self.conv[:-1])):
            nn.init.kaiming_normal_(
                self.conv[i].weight.data, nonlinearity='relu')

        # batch normalization
        self.bn = nn.ModuleList()
        self.bn.extend([nn.BatchNorm2d(C, C) for _ in range(D)])
        # initialize the weights of the Batch normalization layers
        for i in range(D):
            nn.init.constant_(self.bn[i].weight.data, 1.25 * np.sqrt(C))

    def forward(self, x):
        D = self.D
        h = F.relu(self.conv[0](x))
        for i in range(D):
            h = F.relu(self.bn[i](self.conv[i+1](h)))
        y = self.conv[D+1](h) + x
        return y


class UDnCNN(NNRegressor):

    def __init__(self, D, C=64):
        super(UDnCNN, self).__init__()
        self.D = D

        # convolution layers
        self.conv = nn.ModuleList()
        self.conv.append(nn.Conv2d(3, C, 3, padding=1))
        self.conv.extend([nn.Conv2d(C, C, 3, padding=1) for _ in range(D)])
        self.conv.append(nn.Conv2d(C, 3, 3, padding=1))
        # apply He's initialization
        for i in range(len(self.conv[:-1])):
            nn.init.kaiming_normal_(
                self.conv[i].weight.data, nonlinearity='relu')

        # batch normalization
        self.bn = nn.ModuleList()
        self.bn.extend([nn.BatchNorm2d(C, C) for _ in range(D)])
        # initialize the weights of the Batch normalization layers
        for i in range(D):
            nn.init.constant_(self.bn[i].weight.data, 1.25 * np.sqrt(C))

    def forward(self, x):
        D = self.D
        h = F.relu(self.conv[0](x))
        h_buff = []
        idx_buff = []
        shape_buff = []
        for i in range(D//2-1):
            shape_buff.append(h.shape)
            h, idx = F.max_pool2d(F.relu(self.bn[i](self.conv[i+1](h))),
                                  kernel_size=(2, 2), return_indices=True)
            h_buff.append(h)
            idx_buff.append(idx)
        for i in range(D//2-1, D//2+1):
            h = F.relu(self.bn[i](self.conv[i+1](h)))
        for i in range(D//2+1, D):
            j = i - (D // 2 + 1) + 1
            h = F.max_unpool2d(F.relu(self.bn[i](self.conv[i+1]((h+h_buff[-j])/np.sqrt(2)))),
                               idx_buff[-j], kernel_size=(2, 2), output_size=shape_buff[-j])
        y = self.conv[D+1](h) + x
        return y


class DUDnCNN(NNRegressor):

    def __init__(self, D, C=64):
        super(DUDnCNN, self).__init__()
        self.D = D

        # compute k(max_pool) and l(max_unpool)
        k = [0]
        k.extend([i for i in range(D//2)])
        k.extend([k[-1] for _ in range(D//2, D+1)])
        l = [0 for _ in range(D//2+1)]
        l.extend([i for i in range(D+1-(D//2+1))])
        l.append(l[-1])

        # holes and dilations for convolution layers
        holes = [2**(kl[0]-kl[1])-1 for kl in zip(k, l)]
        dilations = [i+1 for i in holes]

        # convolution layers
        self.conv = nn.ModuleList()
        self.conv.append(
            nn.Conv2d(3, C, 3, padding=dilations[0], dilation=dilations[0]))
        self.conv.extend([nn.Conv2d(C, C, 3, padding=dilations[i+1],
                                    dilation=dilations[i+1]) for i in range(D)])
        self.conv.append(
            nn.Conv2d(C, 3, 3, padding=dilations[-1], dilation=dilations[-1]))
        # apply He's initialization
        for i in range(len(self.conv[:-1])):
            nn.init.kaiming_normal_(
                self.conv[i].weight.data, nonlinearity='relu')

        # batch normalization
        self.bn = nn.ModuleList()
        self.bn.extend([nn.BatchNorm2d(C, C) for _ in range(D)])
        # initialize the weights of the Batch normalization layers
        for i in range(D):
            nn.init.constant_(self.bn[i].weight.data, 1.25 * np.sqrt(C))

    def forward(self, x):
        D = self.D
        h = F.relu(self.conv[0](x))
        h_buff = []

        for i in range(D//2 - 1):
            torch.backends.cudnn.benchmark = True
            h = self.conv[i+1](h)
            torch.backends.cudnn.benchmark = False
            h = F.relu(self.bn[i](h))
            h_buff.append(h)

        for i in range(D//2 - 1, D//2 + 1):
            torch.backends.cudnn.benchmark = True
            h = self.conv[i+1](h)
            torch.backends.cudnn.benchmark = False
            h = F.relu(self.bn[i](h))

        for i in range(D//2 + 1, D):
            j = i - (D//2 + 1) + 1
            torch.backends.cudnn.benchmark = True
            h = self.conv[i+1]((h + h_buff[-j]) / np.sqrt(2))
            torch.backends.cudnn.benchmark = False
            h = F.relu(self.bn[i](h))

        y = self.conv[D+1](h) + x
        return y

In [23]:
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch

class XYZ(Dataset):
    def __init__(self, folder_path, img_size=(256, 256)):  # Corrected __init__
        self.folder_path = folder_path
        self.img_size = img_size
        self.image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) 
                            if f.lower().endswith(('jpg', 'jpeg', 'png', 'bmp', 'tiff'))]

        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize between [-1,1]
        ])

    def __len__(self):  # Corrected __len__
        return len(self.image_paths)

    def __getitem__(self, idx):  # Corrected __getitem__
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")  
        image = self.transform(image)
        return image, img_path.replace("/kaggle/input/new-noise", "")

def get_dataloader(folder_path, batch_size=32, shuffle=True, num_workers=4):
    dataset = XYZ(folder_path)  # Now works correctly
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
    return dataloader

# Example usage
loader = get_dataloader("/kaggle/input/new-noise", 8)


In [24]:
 net = DUDnCNN(6, 64)
checkpoint = torch.load("/kaggle/input/denoise/pytorch/default/1/checkpoint.pth.tar")

# Step 3: Restore the state
net.load_state_dict(checkpoint['Net'])


<ipython-input-24-6812d9ef5d62>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/kaggle/input/denoise/pytorch/default/1/checkpoint.pth.tar")


<All keys matched successfully>

In [19]:
from torchvision.transforms import ToPILImage

def save_batch_as_png(tensor_batch, output_paths):
    batch_size = tensor_batch.shape[0]

    if tensor_batch.ndim != 4 or tensor_batch.shape[1] != 3:
        raise ValueError("Tensor batch must have shape (batch_size, 3, 256, 256)")
    
    if len(output_paths) != batch_size:
        raise ValueError(f"Number of output paths ({len(output_paths)}) must match batch size ({batch_size})")

    for i in range(batch_size):
        tensor = tensor_batch[i]

        if tensor.min() < 0 or tensor.max() <= 1:
            tensor = (tensor * 0.5) + 0.5 

        tensor = torch.clamp(tensor, 0, 1)
        image = ToPILImage()(tensor)
        image.save( os.path.join("/kaggle/working/output" , output_paths[i]) )
        print(f"Saved image {i + 1}/{batch_size} to {output_paths[i]}")
        

In [ ]:
#pwd

In [20]:
#for itm , name in loader:

   # out = net(itm)
   # save_batch_as_png(out  , name)
    

Saved image 1/8 to /0977.png
Saved image 2/8 to /0951.png
Saved image 3/8 to /0000011.png
Saved image 4/8 to /0000047.png
Saved image 5/8 to /0917.png
Saved image 6/8 to /0952.png
Saved image 7/8 to /0962.png
Saved image 8/8 to /0000016.png
Saved image 1/8 to /0907.png
Saved image 2/8 to /0902.png
Saved image 3/8 to /0000044.png
Saved image 4/8 to /0964.png
Saved image 5/8 to /0000013.png
Saved image 6/8 to /0000001.png
Saved image 7/8 to /0000039.png
Saved image 8/8 to /0000012.png
Saved image 1/8 to /0908.png
Saved image 2/8 to /0982.png
Saved image 3/8 to /0959.png
Saved image 4/8 to /0901.png
Saved image 5/8 to /0934.png
Saved image 6/8 to /0000096.png
Saved image 7/8 to /0000020.png
Saved image 8/8 to /0950.png
Saved image 1/8 to /0000050.png
Saved image 2/8 to /0000067.png
Saved image 3/8 to /0000087.png
Saved image 4/8 to /0970.png
Saved image 5/8 to /0996.png
Saved image 6/8 to /0938.png
Saved image 7/8 to /0000098.png
Saved image 8/8 to /0926.png
Saved image 1/8 to /0928.png
S

In [21]:
ls

In [ ]:
'''import zipfile
import os

def zip_only_png(folder_path, output_zip_path):
    with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                if file.lower().endswith('.png'): 
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, start=folder_path)  
                    zipf.write(file_path, arcname)
                    print(f"Added: {file_path}")
    
    print(f"Zipped PNG files saved at: {output_zip_path}")
zip_only_png("/kaggle/working/output", "/kaggle/working/out.zip")
'''

In [22]:
#cd /kaggle/working

/kaggle/working


In [26]:
class Args():
    def __init__(self):
        self.output_dir = 'checkpoints1/'
        self.num_epochs = 13
        self.D = 6
        self.C = 64
        self.plot = True
        self.model = 'dudncnn'
        self.lr = 1e-4
        self.image_size = (256, 256)
        self.test_image_size = (256, 256)
        self.batch_size = 16
        self.sigma = 30

In [ ]:
import argparse
import torch
import matplotlib.pyplot as plt


def run(args):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'


    # model
    if args.model == 'dncnn':
        net = DnCNN(args.D, C=args.C).to(device)
    elif args.model == 'udncnn':
        net = UDnCNN(args.D, C=args.C).to(device)
    elif args.model == 'dudncnn':
        net = DUDnCNN(args.D, C=args.C).to(device)
    else:
        raise NameError('Please enter: dncnn, udncnn, or dudncnn')

    # optimizer
    adam = torch.optim.Adam(net.parameters(), lr=args.lr)

    # stats manager
    stats_manager = DenoisingStatsManager()

    # experiment
    exp =  Experiment(net, train_subset, val_subset, adam, stats_manager, batch_size=args.batch_size,
                        output_dir=args.output_dir, perform_validation_during_training=True)

    # run
    if args.plot:
        fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(9, 7))
        exp.run(num_epochs=args.num_epochs, plot=lambda exp: plot(exp, fig=fig, axes=axes,
                                                noisy=val_subset[73][0]))
    else:
        exp.run(num_epochs=args.num_epochs)


if __name__ == '__main__':
    args = Args()
    print(args)
    run(args)

Start/Continue training from epoch 0
Length of train loader is 3399


In [ ]:
'''import matplotlib.pyplot as plt
import cv2

def plot_image(image_path):
    """Reads an image from the given path and displays it."""
    img = cv2.imread(image_path)  # Read the image
    if img is None:
        print("Error: Unable to read image. Check the file path.")
        return
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB (OpenCV loads images in BGR)
    
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.axis("off")  # Hide axes
    plt.show()

# Example usage
# plot_image("path/to/your/image.jpg")
''''''

In [ ]:
plot_image("/kaggle/input/image-dataset-collection/HR/HR/train/0001000/0000001.png")

In [ ]:
plot_image("/kaggle/input/image-dataset-collection/train/train/0001000/0000001x4.png")

In [23]:
list_of_files = os.listdir("/kaggle/input/new-noise")
folder_path = "/kaggle/input/new-noise"

In [24]:
transform_out =  transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize between [-1,1]
        ])

In [30]:
for image_path in list_of_files:
    image = Image.open(os.path.join(folder_path, image_path)).convert("RGB")  
    transform = transforms.Compose([
        transforms.ToTensor() 
    ])
    image_tensor = transform(image)
    original_resolution =list( image_tensor.shape )[::-1]
    

    in_tensor = transform_out(image).unsqueeze(dim = 0)
    out = net(in_tensor)
    save_tensor_as_png(out.squeeze(dim = 0) ,os.path.join("/kaggle/working/out_img1" , image_path) ,   original_resolution[:-1] )
    

Saved image to /kaggle/working/out_img1/0000089.png with resolution [1152, 864]
Saved image to /kaggle/working/out_img1/0000014.png with resolution [872, 584]
Saved image to /kaggle/working/out_img1/0000026.png with resolution [1280, 848]
Saved image to /kaggle/working/out_img1/0968.png with resolution [2040, 1272]
Saved image to /kaggle/working/out_img1/0947.png with resolution [2040, 1040]
Saved image to /kaggle/working/out_img1/1000.png with resolution [2040, 1368]
Saved image to /kaggle/working/out_img1/0950.png with resolution [2040, 1352]
Saved image to /kaggle/working/out_img1/0971.png with resolution [2040, 1352]
Saved image to /kaggle/working/out_img1/0940.png with resolution [2040, 1320]
Saved image to /kaggle/working/out_img1/0000056.png with resolution [1520, 1016]
Saved image to /kaggle/working/out_img1/0000019.png with resolution [752, 1008]
Saved image to /kaggle/working/out_img1/0000030.png with resolution [1920, 1368]
Saved image to /kaggle/working/out_img1/0000021.png

In [28]:
import torch
from torchvision.transforms import ToPILImage

import torch
from torchvision.transforms import ToPILImage
from PIL import Image

def save_tensor_as_png(tensor, file_path="output.png", resolution=(256, 256)):
    """
    Save a tensor as a PNG image with a specified resolution.

    Args:
        tensor (torch.Tensor): Tensor of shape (3, H, W).
        file_path (str): File path to save the image.
        resolution (tuple): Target resolution (width, height) in pixels.
    """
    # Validate tensor shape
    if tensor.ndim != 3 or tensor.shape[0] != 3:
        raise ValueError("Tensor must have shape (3, H, W)")

    # Denormalize if necessary (from [-1, 1] to [0, 1])
    if tensor.min() < 0 or tensor.max() <= 1:
        tensor = (tensor * 0.5) + 0.5

    # Clamp values to [0, 1] range
    tensor = torch.clamp(tensor, 0, 1)

    # Convert to PIL image
    image = ToPILImage()(tensor)

    # Resize to specified resolution
    image = image.resize(resolution, Image.LANCZOS)

    # Save the image
    image.save(file_path)
    print(f"Saved image to {file_path} with resolution {resolution}")
    

In [29]:
mkdir out_img1

In [31]:
#import shutil

#shutil.make_archive("/kaggle/working/team14", 'zip', "/kaggle/working/out_img1")

'/kaggle/working/team14.zip'